In [ ]:
!pip install streamlit
!pip install pandas
!pip install altair
!pip install numpy
!pip install matplotlib
!pip install vega_datasets
!pip install pydeck
!pip install xlrd==2.0.1 
!pip install plotly

In [4]:
# Loading the dataset from drive to colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%%writefile app.py

import streamlit as st
import pandas as pd
import altair as alt
import numpy as np
import matplotlib.pyplot as plt
from vega_datasets import data
import pydeck as pdk
import xlrd
import plotly.express as px

option = st.sidebar.radio( "Select the dashboard", ('Total Number of Incidents Per Ward', 'Total Number of Incidents Per District', 'Total Number of Incidents Per Community Area', 'Map Visualization - Number of crimes at a given time of a day', 'Map Visualization - Locations with most theft/burglaries'))

@st.cache(persist=True)
def load_data():
    data =  pd.read_csv("drive/My Drive/Crimes_dataset.csv", nrows=100000)
    # data =  pd.read_csv("drive/My Drive/Crimes_dataset.csv")
    data.dropna(subset=['Latitude','Longitude'],inplace=True)
    data['time'] = pd.to_datetime(data['Date'].astype(str)).dt.hour
    data['date']=[d.date() for d in pd.to_datetime(data['Date'].astype(str))]
    data['day']=[d.day_name() for d in pd.to_datetime(data['Date'].astype(str))]
   
    data.dropna(subset=['Latitude','Longitude'],inplace=True)
    data.drop(columns=['Date'],inplace=True)
    lowercase=lambda x: str(x).lower()
    data.rename(lowercase,axis="columns",inplace=True)
    return data

if option == 'Total Number of Incidents Per Ward':
  data =  pd.read_csv("drive/My Drive/Crimes_dataset.csv")
  data=data.dropna()
  data.head()
  data[['District', 'Ward','Community Area']] = data[['District', 'Ward','Community Area']].astype('int')
  data[['District', 'Ward','Community Area']] = data[['District', 'Ward','Community Area']].astype('str')
  data.columns= data.columns.str.strip().str.lower().str.replace(' ','_')
  WardData = pd.DataFrame(data['ward'].value_counts(ascending=True).astype(float))
  WardData = WardData.reset_index()
  WardData.columns = ['ward', 'Crime_Count']
  
  bar_chart1=alt.Chart(WardData,title='Total Number of Incidents Per Ward').mark_bar().encode(

     x=alt.X('ward:N', title='ward'),
    y=alt.Y('Crime_Count:Q', title='Crime_Count'),
  ).properties(
             width=800, height=600
  )
  st.write(bar_chart1)

elif option == 'Total Number of Incidents Per District':
  data = pd.read_csv('drive/My Drive/Crimes_dataset.csv')
  data=data.dropna()
  data.head()
  data[['District', 'Ward','Community Area']] = data[['District', 'Ward','Community Area']].astype('int')
  data[['District', 'Ward','Community Area']] = data[['District', 'Ward','Community Area']].astype('str')
  data.columns= data.columns.str.strip().str.lower().str.replace(' ','_')
  DisData = pd.DataFrame(data['district'].value_counts(ascending=True).astype(float))
  DisData = DisData.reset_index()


  DisData.columns = ['district', 'Crime_Count']

  bar_chart2=alt.Chart(DisData,title='Total Number of Incidents Per District').mark_bar().encode(

     x=alt.X('district:N', title='district'),
    y=alt.Y('Crime_Count:Q', title='Crime_Count'),
  ).properties(
             width=700, height=500
  )
  st.write(bar_chart2)

elif option == 'Total Number of Incidents Per Community Area':

  data = pd.read_csv('drive/My Drive/Crimes_dataset.csv', nrows= 100000)
  # alt.data_transformers.disable_max_rows()
  data2 = data[data["Primary Type"].isin(['THEFT', 'BATTERY', 'CRIMINAL DAMAGE'])]
  select = alt.selection(type='interval')
  values = alt.Chart(data2).mark_point().encode(
      x='Year:N',
      y='Community Area:Q',
      color=alt.condition(select, 'Primary Type:N', alt.value('lightgray'))
  ).add_selection(
      select
  )
  bars = alt.Chart(data2).mark_bar().encode(
      y='Primary Type:N',
      color='Primary Type:N',
      x='count(Primary Type):Q'
  ).transform_filter(
      select
  )
  layered = alt.Chart(data2).mark_bar(opacity=0.7).encode(
      x='Year:O',
      y=alt.Y('Community Area:Q', stack=None),
      color="Primary Type",
  )

  st.write(values | bars)
  # st.write()
  st.write(layered)

elif option == 'Map Visualization - Number of crimes at a given time of a day':
  data=load_data()
  st.header("Number of crimes at a given time of a day")
  hour_slider_one=st.slider("Select hour for specific time visualization",0,23)
  hour_slider_one
  original_data=data
  data=data[data["time"]==hour_slider_one]
  midpoint=(np.average(data['latitude']),np.average(data['longitude']))
  st.markdown("Crime visualization depending on location")

  st.write(pdk.Deck(
      map_style="mapbox://styles/mapbox/light-v10",
      initial_view_state={
          "latitude":midpoint[0],
          "longitude":midpoint[1],
          "zoom":11,
          "pitch":30,
      },
      layers=[
          pdk.Layer(
          "HexagonLayer",
          data=data[['time','latitude','longitude']],
          get_position=["longitude","latitude"],
          auto_highlight=True,
          radius=100,
          extruded=True,
          pickable=True,
          elevation_scale=4,
          elevation_range=[0,1000],
          ),
      ],
  ))


elif option == 'Map Visualization - Locations with most theft/burglaries':
  data=load_data()
  st.header("Locations with most theft/burglaries")
  hour_slider_two=st.slider("Select hour for specific time visualization",0,23)
  hour_slider_two
  data_two=data[data["primary type"]=="THEFT"]
  data_three=data_two[data_two["time"]==hour_slider_two]
  midpoint=(np.average(data_three['latitude']),np.average(data_three['longitude']))
  st.markdown("Theft visualization depending on location")

  st.write(pdk.Deck(
      map_style="mapbox://styles/mapbox/light-v10",
      initial_view_state={
          "latitude":midpoint[0],
          "longitude":midpoint[1],
          "zoom":11,
          "pitch":30,
      },
      layers=[
          pdk.Layer(
          "HexagonLayer",
          data=data_three[['time','latitude','longitude']],
          get_position=["longitude","latitude"],
          auto_highlight=True,
          radius=100,
          extruded=True,
          pickable=True,
                    elevation_scale=4,
          elevation_range=[0,1000],
          ),
      ],
  ))

Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

2022-04-29 02:25:49.045 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 3.412s
your url is: https://pretty-falcon-50.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.74.193.174:8501

